In [1]:
import datetime

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

# # For pre-trained embeddings:
# import tensorflow_hub as hub
import tensorflow_text as text

# %load_ext tensorboard

tf.__version__

'2.3.2'

In [2]:
data = pd.read_csv("/home/alexander/projects/car_data_analysis/repeat_mini_proj_1/all_ads_w_omit_and_ranger.csv")
data.head()

,Unnamed: 0,mileage,manufactured.year,engine.size,owners,desc,price,make,id,age,ranger_predict
0,1,149000.0,2003,1.6,8,"12mths mot, good tyres and just been serviced ...",800,Volkswagen,aa8fd5ada99a4098beca3c64ab7a62e5,18,1100.019033
1,2,115000.0,2005,1.6,5,Spares or Repair! My partners lovely car just ...,750,Volkswagen,e2c0072eed0e4ac7b81b8d6fdb60c37b,16,1173.489300
2,3,61000.0,2003,1.4,4,Low mileage car with working electric roof. Pi...,395,Citroen,84145e24c05b489984dded1a4af6454d,18,700.731300
3,4,65659.0,2005,1.6,5,A fun covertable car. Fair size boot. Selling ...,500,Ford,bbbd3061dfe84638a772d1abdd210824,16,1038.943733
4,5,143510.0,2004,1.6,5,Bought this direct from my mechanic a year ago...,800,MINI,d22536078bb842edbee16f654462b580,17,1205.553133


In [3]:
max((len(x) for x in data["id"]))

32

In [4]:
data["price"] = data["price"].astype(np.float32)
data["desc"] = data["desc"].astype("string")
data["id"] = data["id"].astype("string")

train, test = train_test_split(data, test_size=0.3)

In [5]:
train.head()

,Unnamed: 0,mileage,manufactured.year,engine.size,owners,desc,price,make,id,age,ranger_predict
4417,6890,6814.0,2014,1.0,2,"Very low mileage. Black, 2 owners, £4,980",4980.0,Smart,cbc23616194f4353afc0923be9c0ed58,7,5395.644700
23312,38204,86000.0,2011,2.2,4,"2011,Price includes private number plate worth...",5250.0,Jeep,ca4d2e24d87e42b8b2ab6f0106c7fd81,10,5096.558767
22003,36158,115000.0,2014,1.7,1,This Hyundai is being offered for almost 2 tho...,3999.0,Hyundai,84c926f2bb9d4c178b111bdf7fb93a39,7,4636.961667
14227,22153,43000.0,2016,1.0,1,"Recent MOT, Full service history White, 1 owne...",4205.0,Citroen,4fd75455117c4ff288cb9c317643cf45,5,4743.564767
23666,38792,71830.0,2013,2.2,7,*** NIL DEPOSIT FINANCE AVAILABLE ***. PRIVATE...,5699.0,Vauxhall,9518e53803294ca2876ae17569c5a860,8,5440.858633


In [6]:
test.head()

,Unnamed: 0,mileage,manufactured.year,engine.size,owners,desc,price,make,id,age,ranger_predict
1581,2441,35000.0,2016,1.2,2,"Smoke free, Pet free. Free road tax., Recent M...",5150.0,Citroen,1bd3d7648016450ca628e73a7efdca98,5,5621.969633
3758,5906,103000.0,2007,2.0,5,** WINTER SALE - 200 OFF ** Very smart and spo...,2795.0,Volvo,fbdcb581443a4932baa1951bf04d462b,14,2618.254500
5136,8093,88500.0,2011,2.0,3,Priced for a quick sale. Metallic black Sciroc...,5250.0,Volkswagen,a1c52dfd85524dd7a7f53ce2b6ff211c,10,5829.321733
25124,41134,54473.0,2016,1.6,2,Full service history 12 months MOT 12 months ...,8800.0,Peugeot,ccde675a06904140b706f512b324567c,5,7864.017767
12347,19614,77760.0,2007,1.2,3,"Fiat Panda Dynamic 2007, Full service history,...",1150.0,Fiat,e17c434dfcd34dc787fe96a133532494,14,1053.487933


In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((train["desc"].values, train["price"].values))
test_ds = tf.data.Dataset.from_tensor_slices((test["desc"].values, test["price"].values))

train_ds = train_ds.shuffle(10000).batch(64)
test_ds = test_ds.batch(64)

In [8]:
from tensorflow.keras.layers import Input, Dense

VOCAB_SIZE = 5000

text_input = Input(shape=(), dtype=tf.string, name='text')
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE,output_mode='tf-idf')
encoder.adapt(train_ds.map(lambda text, label: text))
net = encoder(text_input)
net = Dense(64, activation="relu")(net)
net = Dense(32, activation="relu")(net)
net = Dense(1, activation="relu")(net)

model = tf.keras.Model(text_input, net)
model.compile(loss='mse')

In [9]:
# Bug with tensorboard and some experimental layers:
# https://github.com/tensorflow/tensorboard/issues/4530
# ! rm -rf ./logs/

# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1, restore_best_weights=True)

history = model.fit(train_ds,
    epochs=100,
    validation_data=test_ds,
    callbacks=[early_stopping])

Epoch 1/100
332/332 [==============================] - 33s 100ms/step - loss: 19851486.0000 - val_loss: 12993357.0000
Epoch 2/100
332/332 [==============================] - 33s 99ms/step - loss: 10801203.0000 - val_loss: 9345663.0000
Epoch 3/100
332/332 [==============================] - 36s 108ms/step - loss: 7403006.5000 - val_loss: 6658849.5000
Epoch 4/100
332/332 [==============================] - 35s 106ms/step - loss: 5243096.0000 - val_loss: 4947620.5000
Epoch 5/100
332/332 [==============================] - 31s 94ms/step - loss: 3765312.0000 - val_loss: 3790617.0000
Epoch 6/100
332/332 [==============================] - 30s 90ms/step - loss: 2801972.0000 - val_loss: 3117643.7500
Epoch 7/100
332/332 [==============================] - 31s 93ms/step - loss: 2238307.0000 - val_loss: 2721268.0000
Epoch 8/100
332/332 [==============================] - 35s 105ms/step - loss: 1897771.3750 - val_loss: 2486816.7500
Epoch 9/100
332/332 [==============================] - 33s 100ms/step - l

In [10]:
model.evaluate(test_ds)

143/143 [==============================] - 10s 73ms/step - loss: 1761321.2500


1761321.25

In [11]:
data_ds = tf.data.Dataset.from_tensor_slices((data["desc"].values, data["price"].values))
data_ds = data_ds.batch(64)

In [12]:
model.evaluate(data_ds)

474/474 [==============================] - 35s 73ms/step - loss: 946794.6250


946794.625

In [13]:
nlp_preds = model.predict(data_ds)

In [14]:
data["nlp_pred"] = nlp_preds
data.head()

,Unnamed: 0,mileage,manufactured.year,engine.size,owners,desc,price,make,id,age,ranger_predict,nlp_pred
0,1,149000.0,2003,1.6,8,"12mths mot, good tyres and just been serviced ...",800.0,Volkswagen,aa8fd5ada99a4098beca3c64ab7a62e5,18,1100.019033,893.662842
1,2,115000.0,2005,1.6,5,Spares or Repair! My partners lovely car just ...,750.0,Volkswagen,e2c0072eed0e4ac7b81b8d6fdb60c37b,16,1173.489300,729.995728
2,3,61000.0,2003,1.4,4,Low mileage car with working electric roof. Pi...,395.0,Citroen,84145e24c05b489984dded1a4af6454d,18,700.731300,935.655090
3,4,65659.0,2005,1.6,5,A fun covertable car. Fair size boot. Selling ...,500.0,Ford,bbbd3061dfe84638a772d1abdd210824,16,1038.943733,2770.693115
4,5,143510.0,2004,1.6,5,Bought this direct from my mechanic a year ago...,800.0,MINI,d22536078bb842edbee16f654462b580,17,1205.553133,815.074646


In [15]:
sample = data.iloc[2]["desc"]
sample = tf.constant(sample)
model.predict(sample[tf.newaxis, ...])

array([[935.6549]], dtype=float32)

In [16]:
real, nlp, ranger = data[["price"]].values,data[["nlp_pred"]].values, data[["ranger_predict"]].values
norm = np.random.normal(0, np.std(nlp-ranger), size=real.shape)
max(real),max(nlp), max(ranger)

(array([10000.], dtype=float32),
 array([16252.152], dtype=float32),
 array([9722.67296667]))

In [ ]:
BINS = 50
fig = plt.figure(figsize=(14,7))
gs = fig.add_gridspec(2, 6)
ax0 = fig.add_subplot(gs[0,0:2])
ax1 = fig.add_subplot(gs[0,2:4])
ax2 = fig.add_subplot(gs[0,4:6])
ax3 = fig.add_subplot(gs[1,1:3])
ax4 = fig.add_subplot(gs[1,3:5])


_, b, _ = ax0.hist(real, bins=BINS, alpha=0.3, label="real")
ax0.hist(ranger, bins=b, alpha=0.3, label="ranger")
ax0.hist(nlp, bins=b, alpha=0.3, label="tfidf")
ax0.legend()
ax0.set_title("Comparing predicted distribution shapes")

ax1.hist(real-ranger, bins=BINS, density=True, alpha=0.4, label="empirical dist")
ax1.hist(norm, bins=BINS, density=True, alpha=0.4, label="normal matched variance")
ax1.legend()
ax1.set_title("Analysing the distribution of (tfidf - ranger) predictions")

ax2.scatter(real, nlp, label="tfidf", alpha=0.3, s=1)
ax2.scatter(real, ranger, label="ranger", alpha=0.3, s=2)
ax2.plot(real, real, label="truth", color="black")
ax2.legend()
ax2.set_title("Overlayed confusion")
ax2.set_ylabel("Prediction")
ax2.set_xlabel("Truth")

ax3.hist2d(real[:,0], nlp[:,0], label="tfidf", bins=(50,50))
ax3.plot(real, real, label="truth", color="red")
ax3.legend()
ax3.set_title("tfidf confusion density")
ax3.set_ylabel("Prediction")
ax3.set_xlabel("Truth")

ax4.hist2d(real[:,0], ranger[:,0], label="ranger", bins=(50,50))
ax4.plot(real, real, label="truth", color="red")
ax4.legend()
ax4.set_title("ranger confusion density")
ax4.set_ylabel("Prediction")
ax4.set_xlabel("Truth")

fig.tight_layout(pad=3)

# Plot predictions against inputs?
# Somehow get the last confusion plot as a 3D hist
# Differentiate test from train
# Parameterized step function on regression?
# Petrol station transaction - Gordon Blunt, Hand, Adams (?)
# - Look into citations
# match axes on density plots
# Is there a way to combine them that makes them better?
# Shows the description has good consistent signal
# Look into visualising investigating - check the numbers isn't being read (knock out the numbers)
# Look into rescaling data
# Look at outliers
# R squared
# MSE (test)
# MAE (test)
# Correlation/beta of the confusion
# Standardize histogram bins for better comparison

In [ ]:
''